# soilgrids

This package provides a minimal wrapper for the ISRIC Soilgrids API, allowing 
users to query soil properties by latitude/longitude and to perform basic 
analyses on the returned data.

Functions:
*   `get_soilgrids()`: Provides a simple wrapper for the /properties/query
    API endpoint, parsing the geojson response into a pandas DataFrame.
        
Classes:
*   `SoilGrids()`: Provides methods for reading data from Soilgrids and basic 
    including utilities for aggregating and analysing the returned data.

Useful links:
*   Documentation for the API: <https://rest.isric.org/soilgrids/v2.0/docs>
*   ISRIC REST entry page, including fair use policy: <https://rest.isric.org>
*   ISRIC data and software policy: <https://www.isric.org/about/data-policy>
*   Soilgrids FAQ: <https://www.isric.org/explore/soilgrids/faq-soilgrids>

## Querying data using `SoilGrids`:
The following code reads in the mean values for clay, sand, silt and ocs 
(organic carbon stock) in the top 30cm of soil for a random set of 50 points 
within roughly 25km of 
[Herning, Denmark](https://en.wikipedia.org/wiki/Herning). Note that points can 
be queried at a maximum rate of 5/minute, so the following code takes about 10 
minutes to run:

In [1]:
# ~~~ Console messages disabled for a cleaner output ~~~ #
import logging
logging.getLogger('soilgrids').setLevel(logging.ERROR)
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #

from soilgrids import SoilGrids
import pandas as pd


sg = SoilGrids()

sg.get_points_sample(
    50,
    lat_min=56.225297, lat_max=55.958103,
    lon_min=8.662215, lon_max=9.354390,
    soil_property=['clay', 'sand', 'silt', 'ocs'],
    depth=['0-5cm', '5-15cm', '15-30cm', '0-30cm'],
    value='mean'
)

sg.data \
    .filter([
        'lat', 'lon', 'soil_property', 'mapped_units', 
        'target_units', 'depth', 'mean'
    ])

NameError: name 'sg' is not defined

## Get the property (clay, sand, silt) with the highest value for each point
The `SoilGrids` class provides a handy utility `main_properties()` for finding
the most abundant soil type (i.e. property) for each point. In this case, we
see that the Herning region is quite sandy:

In [11]:
sg.main_properties()

,lat,lon,soil_property
0,56.235719,9.227313,sand
1,56.237697,8.984183,sand
2,56.239461,9.191985,sand
3,56.255252,9.049602,sand
4,56.259932,8.923327,sand
5,56.262672,9.002590,sand
6,56.264488,9.004942,sand
7,56.267499,9.058695,sand
8,56.273890,9.107779,sand
9,56.297945,9.109309,sand


## Relationship between clay, sand, silt and organic carbon stock
The `ocs_correlation()` method fits and displays summary statistics for a linear 
model using sand, clay and silt as predictors and OCS as the response variable. 
In this case, we find that these properties are not particularly correlated 
with OCS.

In [18]:
print(sg.ocs_correlation(capture_output=True))


Call:
lm(formula = clay + sand + silt ~ ocs, data = input_data)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.07700 -1.05049 -0.05614  0.92996  1.94212 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  1.000e+03  1.337e+00 748.061   <2e-16 ***
ocs         -1.738e-03  2.387e-02  -0.073    0.942    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.8758 on 48 degrees of freedom
Multiple R-squared:  0.0001104,	Adjusted R-squared:  -0.02072 
F-statistic: 0.005302 on 1 and 48 DF,  p-value: 0.9423




## Disclaimers

*   Use of this package is subject to [ISRIC data and software policy](https://www.isric.org/about/data-policy).

*   This package is licensed as [GPL-2](LICENSE).